In [4]:
import random
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import pydot

from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus
import score_solution
from sklearn.metrics import cohen_kappa_score, make_scorer



train_data = pd.read_csv('all_train_data.csv')
test_data = pd.read_csv('all_test_data.csv')

adoptionSpeed = train_data['AdoptionSpeed'] 
del train_data['AdoptionSpeed']



In [6]:
test_ids = test_data[['PetID']]

del train_data["Name"]
del train_data["RescuerID"]
del train_data["Description"]
del train_data["PetID"]


del test_data["Name"]
del test_data["RescuerID"]
del test_data["Description"]
del test_data["PetID"]

train_data = train_data.fillna(0)
test_data = test_data.fillna(0)



In [7]:
from sklearn.datasets import load_boston
import xgboost as xgb
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier


gbc_model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1,
     max_depth=2)
agb_model = AdaBoostClassifier(learning_rate=0.1, n_estimators=1000,
                           algorithm='SAMME.R', base_estimator=DecisionTreeClassifier(max_depth=2))
extra_model = ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

rf_model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
bagging_model = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 1000,
                            max_samples=0.5, max_features=0.5)

parameters = {
    'learning_rate':0.01,'max_depth':4,
    'subsample':0.8,'objective':'binary:logistic',
    'num_class':5, 'n_jobs':4,  'n_estimators':1000}

xgb_model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=650,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

classifiers = [
    ('gbc', gbc_model), ('agb', agb_model),
    ('extra', extra_model), ('rf', rf_model),
    ('bagging', bagging_model), ("xgb", xgb_model)
]

voting_model = VotingClassifier(estimators=classifiers, voting='hard')

voting_model.fit(train_data, adoptionSpeed)

ans = voting_model.predict(test_data)

ans


array([2, 4, 4, ..., 1, 2, 4])

In [9]:
submission = pd.DataFrame({'PetID': test_ids['PetID'].values, 'AdoptionSpeed': ans})
print(submission.head())
submission.to_csv('submission.csv', index=False)



       PetID  AdoptionSpeed
0  378fcc4fc              2
1  73c10e136              4
2  72000c4c5              4
3  e147a4b9f              4
4  43fbba852              4
